##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Prompting with an Apollo 11 transcript

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Apollo_11.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

This notebook provides a quick example of how to prompt Gemini using a text file. In this case, you'll use a 400 page transcript from [Apollo 11](https://www.nasa.gov/history/alsj/a11/a11trans.html).

In [ ]:
%pip install -U -q "google-genai>=1.0.0"

### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [ ]:
from google.colab import userdata
from google import genai

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

Download the transcript.

In [ ]:
!wget https://storage.googleapis.com/generativeai-downloads/data/a11.txt

--2025-03-04 11:48:01--  https://storage.googleapis.com/generativeai-downloads/data/a11.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.135.207, 74.125.142.207, 74.125.195.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.135.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 847790 (828K) [text/plain]
Saving to: ‘a11.txt.1’

a11.txt.1           100%[===================>] 827.92K  --.-KB/s    in 0.008s  

2025-03-04 11:48:01 (107 MB/s) - ‘a11.txt.1’ saved [847790/847790]



Upload the file using the File API so its easier to pass it to the model later on.

In [ ]:
text_file_name = "a11.txt"
print(f"Uploading file...")
text_file = client.files.upload(file=text_file_name)
print(f"Completed upload: {text_file.uri}")

Uploading file...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/umcpmw8s1adm


## Generate Content

After the file has been uploaded, you can make `client.models.generate_content` requests that reference the File API URI. Then you will ask the model to find a few lighthearted moments.

In [ ]:
prompt = "Find four lighthearted moments in this text file."

MODEL_ID = "gemini-2.0-flash" # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-flash-preview-04-17","gemini-2.5-pro-exp-03-25"] {"allow-input":true, isTemplate: true}

response = client.models.generate_content(
  model=f"models/{MODEL_ID}",
  contents=[
   prompt,
   text_file
  ],
  config={
   "httpOptions": {"timeout": 600}
  }
)

print(response.text)

Here are four lighthearted moments from the Apollo 11 air-to-ground voice transcription:

1.  **00 00 05 35 CDR: You sure sound clear down there, Bruce. Sounds like you're sitting in your living room.**
    **00 00 05 39 CC: Oh, thank you. You all are coming through beautifully, too.**
    *This exchange is light because the Commander compliments the CapCom on the clarity of his voice, suggesting a very relaxed and comfortable connection.*

2.  **00 00 54 13 CMP: And tell Glenn Parker down at the Cape that he lucked out.**
    **00 00 54 17 CC: Understand. Tell Glenn Parker he lucked out.**
    **00 00 54 22 CMP: Yes. He lucked out. He doesn't owe me a cup of coffee.**
    **00 00 54 26 CC: This is Houston. Roger. We'll pass it on.**
    *This is funny because they are saying they're saying someone "lucked out" and doesn't owe someone a cup of coffee. The phrase "lucked out" has a casual, playful feel, and the quick passing of a message involving coffee adds to it.*

3.  **00 01 29 27 

## Delete File

Files are automatically deleted after 2 days or you can manually delete them using `files.delete()`.

In [ ]:
client.files.delete(name=text_file.name)

DeleteFileResponse()

## Learning more

The File API accepts files under 2GB in size and can store up to 20GB of files per project. Learn more about the [File API](../quickstarts/File_API.ipynb) here.